In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
import datetime
from sklearn.model_selection import GridSearchCV

In [2]:
test = pd.read_csv("../input/tp2grupo3/test.csv")
train = pd.read_csv("../input/tp2grupo3/train.csv")

In [3]:
train['es_luminoso'] = 0
train['tiene_balcon'] = 0
train['es_amplio'] = 0

test['es_luminoso'] = 0
test['tiene_balcon'] = 0
test['es_amplio'] = 0

train = train.fillna(value = {'descripcion': 0})
test = test.fillna(value = {'descripcion': 0})

In [4]:
train.loc[train['descripcion'].str.contains("luminoso")==True, 'es_luminoso'] = 1
train.loc[train['descripcion'].str.contains("amplio")==True, 'tiene_balcon'] = 1
train.loc[train['descripcion'].str.contains("balcon")==True, 'es_amplio'] = 1

test.loc[test['descripcion'].str.contains("luminoso")==True, 'es_luminoso'] = 1
test.loc[test['descripcion'].str.contains("amplio")==True, 'tiene_balcon'] = 1
test.loc[test['descripcion'].str.contains("balcon")==True, 'es_amplio'] = 1

print(train['es_luminoso'].sum())
print(train['tiene_balcon'].sum())
print(train['es_amplio'].sum())

print(test['es_luminoso'].sum())
print(test['tiene_balcon'].sum())
print(test['es_amplio'].sum())

381
37306
4839
94
9325
1232


In [5]:
test['fecha'] = pd.to_datetime(test['fecha'])
test['year'] = test['fecha'].dt.year
test['month']= test['fecha'].dt.month

train['fecha'] = pd.to_datetime(train['fecha'])
train['year'] = train['fecha'].dt.year
train['month']= train['fecha'].dt.month

In [6]:
train = train.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)
test = test.drop(['fecha','titulo','descripcion','direccion','idzona','lat','lng'], axis=1)

ciudades_en_train = train.ciudad.unique()
ciudades_en_test = test.ciudad.unique()

tipodepropiedad_en_train = train.tipodepropiedad.unique()
tipodepropiedad_en_test = test.tipodepropiedad.unique()

provincia_en_train = train.provincia.unique()
provincia_en_test = test.provincia.unique()

In [8]:
train = pd.get_dummies(train, dummy_na=True)
test = pd.get_dummies(test, drop_first=True)

In [9]:
X = train.drop("precio", axis=1)
y = train['precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=1)

In [10]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.7, learning_rate = 0.04,
                max_depth = 9, min_child_weight = 1, n_estimators = 700, nthread = 4, subsample = 0.7, silent = 1)

xg_reg.fit(X,y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/conda/lib/python3.6/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=9, min_child_weight=1, missing=None, n_estimators=700,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [12]:
train.head(10)

for ciudad in ciudades_en_train:
    if ciudad not in ciudades_en_test:
        test['ciudad_'+str(ciudad)] = 0
        
for provincia in provincia_en_train:
    if provincia not in provincia_en_test:
        test['provincia_'+str(provincia)] = 0
        
for tipodepropiedad in tipodepropiedad_en_train:
    if tipodepropiedad not in tipodepropiedad_en_test:
        test['tipodepropiedad_'+str(tipodepropiedad)] = 0

In [ ]:
some_list = ['tipodepropiedad_Apartamento', 'ciudad_Abasolo', 'provincia_Aguascalientes']
for item in some_list:
    test[item] = 0

In [13]:
test = test[list(X.columns.values)]
print(test.shape)
preds = xg_reg.predict(test)

KeyError: "['tipodepropiedad_Apartamento', 'ciudad_Abasolo', 'provincia_Aguascalientes'] not in index"

In [ ]:
preds2 = xg_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds2)

In [ ]:
res = pd.DataFrame(preds, index=test.index, columns=['precio'])

In [ ]:
res2 = res
res2['id'] = test['id']
res2 = res2.reset_index(drop=True)
res2.head()
res2.set_index('id', inplace = True)
#res.columns = ['id', 'target']
#res.set_index('id', inplace = True)
res2.rename(columns={'precio':'target'}, inplace=True)
print(res2.shape)


#res.to_csv("/kaggle/working/submission_xgboost_city_sudoTFIDF_5.csv", header=True)

In [ ]:
res2.to_csv("/kaggle/working/submission_xgboost_city_sudoTFIDF_5.csv", header=True)